In [ ]:
import sqlite3
import pandas as pd
import os
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Definir caminho da pasta Ecommerce_Dataset
base_path = '/content/drive/MyDrive/Ecommerce_Dataset/'

# Verificar se os arquivos existem
print("Arquivos na pasta Ecommerce_Dataset:")
print(os.listdir(base_path))

# Conectar ao banco de dados SQLite (salvo no Google Drive)
db_path = '/content/drive/MyDrive/Ecommerce_Dataset/ecommerce_test.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Habilitar chaves estrangeiras
cursor.execute("PRAGMA foreign_keys = ON;")

# Remover tabelas existentes, se houver
cursor.execute("DROP TABLE IF EXISTS order_items;")
cursor.execute("DROP TABLE IF EXISTS orders;")
cursor.execute("DROP TABLE IF EXISTS products;")
cursor.execute("DROP TABLE IF EXISTS customers;")
conn.commit()

# Executar o script create_tables.sql (removendo CREATE DATABASE e USE)
sql_script = """
-- Tabela de clientes
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(100),
    email VARCHAR(100),
    registration_date DATE,
    city VARCHAR(50),
    state VARCHAR(50)
);

-- Tabela de produtos
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(200),
    category VARCHAR(50),
    price DECIMAL(10,2),
    brand VARCHAR(50)
);

-- Tabela de pedidos
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL(10,2),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

-- Tabela de itens do pedido
CREATE TABLE order_items (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
"""
try:
    cursor.executescript(sql_script)
    print("Tabelas criadas com sucesso!")
except sqlite3.OperationalError as e:
    print(f"Erro ao criar tabelas: {e}")
    raise

# Importar os CSVs para o banco de dados
try:
    customers_df = pd.read_csv(base_path + 'customers.csv')
    customers_df.to_sql('customers', conn, if_exists='append', index=False)
    print("customers.csv importado com sucesso!")
except FileNotFoundError:
    print(f"Erro: customers.csv não encontrado em {base_path}")
    raise

try:
    products_df = pd.read_csv(base_path + 'products.csv')
    products_df.to_sql('products', conn, if_exists='append', index=False)
    print("products.csv importado com sucesso!")
except FileNotFoundError:
    print(f"Erro: products.csv não encontrado em {base_path}")
    raise

try:
    orders_df = pd.read_csv(base_path + 'orders.csv')
    orders_df.to_sql('orders', conn, if_exists='append', index=False)
    print("orders.csv importado com sucesso!")
except FileNotFoundError:
    print(f"Erro: orders.csv não encontrado em {base_path}")
    raise

try:
    order_items_df = pd.read_csv(base_path + 'order_items.csv')
    order_items_df.to_sql('order_items', conn, if_exists='append', index=False)
    print("order_items.csv importado com sucesso!")
except FileNotFoundError:
    print(f"Erro: order_items.csv não encontrado em {base_path}")
    raise

# Confirmar alterações
conn.commit()
print("Dados importados com sucesso para o banco SQLite!")

# Verificar tabelas criadas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tabelas no banco de dados:", cursor.fetchall())

# Testar uma query simples para verificar os dados
query = "SELECT * FROM customers LIMIT 5;"
result = pd.read_sql_query(query, conn)
print("\nExemplo de dados da tabela customers:")
print(result)

# Fechar conexão (opcional, pode manter aberta para queries)
conn.close()

Mounted at /content/drive
Arquivos na pasta Ecommerce_Dataset:
['customers.csv', 'products.csv', 'orders.csv', 'order_items.csv', 'create_tables.sql', 'ecommerce_test.db']
Tabelas criadas com sucesso!
customers.csv importado com sucesso!
products.csv importado com sucesso!
orders.csv importado com sucesso!
order_items.csv importado com sucesso!
Dados importados com sucesso para o banco SQLite!
Tabelas no banco de dados: [('customers',), ('products',), ('orders',), ('order_items',)]

Exemplo de dados da tabela customers:
   customer_id           customer_name                             email  \
0            1  João Guilherme da Cruz       joao-lucassousa@example.org   
1            2   Henrique da Conceição          novaismiguel@example.net   
2            3         Emanuel Sampaio            rfernandes@example.org   
3            4            Aylla Fogaça            vinicius07@example.net   
4            5  Maria Cecília Oliveira  davi-miguelaparecida@example.net   

  registration_da

In [ ]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados
db_path = '/content/drive/MyDrive/Ecommerce_Dataset/ecommerce_test.db'
conn = sqlite3.connect(db_path)

# 1. Receita total por mês nos últimos 6 meses (julho a dezembro de 2024)
query1 = """
SELECT
    strftime('%Y-%m', order_date) AS month,
    ROUND(SUM(total_amount), 2) AS total_revenue
FROM orders
WHERE order_date >= '2024-07-01' AND order_date <= '2024-12-31'
    AND status = 'Entregue'
GROUP BY strftime('%Y-%m', order_date)
ORDER BY month;
"""
result1 = pd.read_sql_query(query1, conn)
print("\n1. Receita total por mês (últimos 6 meses):")
print(result1)

# 2. Top 5 produtos por receita
query2 = """
SELECT
    p.product_name,
    p.category,
    ROUND(SUM(oi.quantity * oi.unit_price), 2) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'Entregue'
GROUP BY p.product_id, p.product_name, p.category
ORDER BY total_revenue DESC
LIMIT 5;
"""
result2 = pd.read_sql_query(query2, conn)
print("\n2. Top 5 produtos por receita:")
print(result2)

# 3. Receita por categoria de produto
query3 = """
SELECT
    p.category,
    ROUND(SUM(oi.quantity * oi.unit_price), 2) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'Entregue'
GROUP BY p.category
ORDER BY total_revenue DESC;
"""
result3 = pd.read_sql_query(query3, conn)
print("\n3. Receita por categoria de produto:")
print(result3)

# 4. Número de clientes únicos por mês
query4 = """
SELECT
    strftime('%Y-%m', order_date) AS month,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM orders
GROUP BY strftime('%Y-%m', order_date)
ORDER BY month;
"""
result4 = pd.read_sql_query(query4, conn)
print("\n4. Clientes únicos por mês:")
print(result4)

# 5. Ticket médio por cliente
query5 = """
SELECT
    c.customer_id,
    c.customer_name,
    ROUND(AVG(o.total_amount), 2) AS avg_ticket
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.status = 'Entregue'
GROUP BY c.customer_id, c.customer_name
ORDER BY avg_ticket DESC
LIMIT 10;  -- Limitado a 10 para visualização
"""
result5 = pd.read_sql_query(query5, conn)
print("\n5. Ticket médio por cliente (top 10):")
print(result5)

# 6. Top 10 clientes por valor total de compras
query6 = """
SELECT
    c.customer_id,
    c.customer_name,
    ROUND(SUM(o.total_amount), 2) AS total_spent
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.status = 'Entregue'
GROUP BY c.customer_id, c.customer_name
ORDER BY total_spent DESC
LIMIT 10;
"""
result6 = pd.read_sql_query(query6, conn)
print("\n6. Top 10 clientes por valor total de compras:")
print(result6)

# 7. Produtos mais vendidos (por quantidade)
query7 = """
SELECT
    p.product_name,
    p.category,
    SUM(oi.quantity) AS total_quantity
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'Entregue'
GROUP BY p.product_id, p.product_name, p.category
ORDER BY total_quantity DESC
LIMIT 10;
"""
result7 = pd.read_sql_query(query7, conn)
print("\n7. Produtos mais vendidos (por quantidade):")
print(result7)

# 8. Média de preço por categoria
query8 = """
SELECT
    category,
    ROUND(AVG(price), 2) AS avg_price
FROM products
GROUP BY category
ORDER BY avg_price DESC;
"""
result8 = pd.read_sql_query(query8, conn)
print("\n8. Média de preço por categoria:")
print(result8)

# Salvar todas as queries em um arquivo .sql
queries = """
-- 1. Receita total por mês nos últimos 6 meses
SELECT strftime('%Y-%m', order_date) AS month, ROUND(SUM(total_amount), 2) AS total_revenue
FROM orders
WHERE order_date >= '2024-07-01' AND order_date <= '2024-12-31' AND status = 'Entregue'
GROUP BY strftime('%Y-%m', order_date)
ORDER BY month;

-- 2. Top 5 produtos por receita
SELECT p.product_name, p.category, ROUND(SUM(oi.quantity * oi.unit_price), 2) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'Entregue'
GROUP BY p.product_id, p.product_name, p.category
ORDER BY total_revenue DESC
LIMIT 5;

-- 3. Receita por categoria de produto
SELECT p.category, ROUND(SUM(oi.quantity * oi.unit_price), 2) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'Entregue'
GROUP BY p.category
ORDER BY total_revenue DESC;

-- 4. Número de clientes únicos por mês
SELECT strftime('%Y-%m', order_date) AS month, COUNT(DISTINCT customer_id) AS unique_customers
FROM orders
GROUP BY strftime('%Y-%m', order_date)
ORDER BY month;

-- 5. Ticket médio por cliente
SELECT c.customer_id, c.customer_name, ROUND(AVG(o.total_amount), 2) AS avg_ticket
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.status = 'Entregue'
GROUP BY c.customer_id, c.customer_name
ORDER BY avg_ticket DESC;

-- 6. Top 10 clientes por valor total de compras
SELECT c.customer_id, c.customer_name, ROUND(SUM(o.total_amount), 2) AS total_spent
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.status = 'Entregue'
GROUP BY c.customer_id, c.customer_name
ORDER BY total_spent DESC
LIMIT 10;

-- 7. Produtos mais vendidos (por quantidade)
SELECT p.product_name, p.category, SUM(oi.quantity) AS total_quantity
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'Entregue'
GROUP BY p.product_id, p.product_name, p.category
ORDER BY total_quantity DESC
LIMIT 10;

-- 8. Média de preço por categoria
SELECT category, ROUND(AVG(price), 2) AS avg_price
FROM products
GROUP BY category
ORDER BY avg_price DESC;
"""
with open('/content/drive/MyDrive/Ecommerce_Dataset/queries.sql', 'w', encoding='utf-8') as f:
    f.write(queries)
print("\nQueries salvas em /content/drive/MyDrive/Ecommerce_Dataset/queries.sql")

# Fechar conexão
conn.close()


1. Receita total por mês (últimos 6 meses):
     month  total_revenue
0  2024-07     4048915.06
1  2024-08     3564469.54
2  2024-09     4013235.64
3  2024-10     4925822.27
4  2024-11     6404684.15
5  2024-12     5796247.70

2. Top 5 produtos por receita:
               product_name     category  total_revenue
0     Premium TV Voluptates  Eletrônicos      719789.57
1   Generic Laptop Adipisci  Eletrônicos      704768.92
2  Basic Fone de Ouvido Non  Eletrônicos      670186.34
3  Standard Laptop Eligendi  Eletrônicos      655956.86
4           Philips TV Eius  Eletrônicos      633626.34

3. Receita por categoria de produto:
        category  total_revenue
0    Eletrônicos    31261615.75
1  Casa & Jardim     9801907.91
2       Esportes     5611973.29
3           Moda     3583155.57
4         Livros     1348274.27

4. Clientes únicos por mês:
      month  unique_customers
0   2024-01              2876
1   2024-02              2620
2   2024-03              2953
3   2024-04              3